### instalar dependencias

In [27]:
! pip install pandas
! pip install requests

### Obtener data

In [28]:
import requests
import json

with open("jobs.json", 'r', encoding='utf-8') as file:
    jobs = json.load(file)

if not jobs:
    response = requests.get("https://www.getonbrd.com/api/v0/search/jobs", params={
        "query": "Backend",
        "per_page": 100,
        "page": 1,
    })

    jobs = response.json()["data"]

    with open("jobs.json", 'w', encoding='utf-8') as file:
        json.dump(jobs, file, ensure_ascii=False, indent=4)



### guardar data

In [29]:
import json

def guardar_en_json(array_de_objetos, nombre_archivo):
    
    with open(nombre_archivo, 'w', encoding='utf-8') as archivo_json:
        json.dump(array_de_objetos, archivo_json, ensure_ascii=False, indent=4)


guardar_en_json(jobs, 'jobs.json')

### Construir data frame

In [30]:
import pandas as pd

def transform_jobs_to_dataframe(jobs):
    # Estructurar los datos en una lista de diccionarios
    structured_data = []
    for job in jobs:
        job_data = {
            "id": job["id"],
            "title": job["attributes"]["title"],
            "description_headline": job["attributes"].get("description_headline", ""),
            "description": job["attributes"].get("description", ""),
            "projects": job["attributes"].get("projects", ""),
            "functions": job["attributes"].get("functions", ""),
            "benefits": job["attributes"].get("benefits", ""),
            "desirable": job["attributes"].get("desirable", ""),
            "remote": job["attributes"]["remote"],
            "remote_modality": job["attributes"].get("remote_modality", ""),
            "remote_zone": job["attributes"].get("remote_zone", ""),
            "countries": job["attributes"]["countries"],
            "lang": job["attributes"]["lang"],
            "category_name": job["attributes"]["category_name"],
            "perks": job["attributes"]["perks"],
            "min_salary": job["attributes"].get("min_salary"),
            "max_salary": job["attributes"].get("max_salary"),
            "published_at": job["attributes"]["published_at"],
            "response_time_min": job["attributes"]["response_time_in_days"].get("min"),
            "response_time_max": job["attributes"]["response_time_in_days"].get("max"),
            "applications_count": job["attributes"]["applications_count"],
            "public_url": job["links"]["public_url"]
        }
        structured_data.append(job_data)
    
    return pd.DataFrame(structured_data)

df_jobs = transform_jobs_to_dataframe(jobs)
df_jobs

,id,title,description_headline,description,projects,functions,benefits,desirable,remote,remote_modality,...,lang,category_name,perks,min_salary,max_salary,published_at,response_time_min,response_time_max,applications_count,public_url
0,desarrollador-a-backend-javase-spring-boot-aws...,Desarrollador/a Back-end Javase Spring Boot AWS,Requerimientos del cargo,<p><strong>¿Qué requerimos?</strong></p><ul><l...,<p><strong>Acerca del empleo<br></strong>Sopho...,<p><strong>Tendrás la oportunidad de:</strong>...,<p><strong>Nosotros te ofrecemos</strong> 💜</p...,<p><strong>Es un Plus:</strong></p><ul><li>Exp...,True,remote_local,...,lang_not_specified,SysAdmin / DevOps / QA,"[wellness, digital_library, health_coverage, c...",NaN,NaN,1716475444,NaN,NaN,13,https://www.getonbrd.com/jobs/desarrollador-a-...
1,tech-lead-arkho-santiago-70da,Data Tech Lead,Perfil del archer,<div>¿Cuáles son los requerimientos del cargo?...,<div>ARKHO es una consultora experta en tecnol...,<div>Estamos en busca de un profesional con ex...,<ul><li>Día administrativo semestral hasta los...,"<ul><li>Back: Microservicios, API devolopment<...",True,fully_remote,...,lang_not_specified,Programming,"[remote_full, flexible_hours, informal_dressco...",NaN,NaN,1716436734,5.0,9.0,172,https://www.getonbrd.com/jobs/tech-lead-arkho-...
2,desarrollador-a-backend-springboot-empresas-sb...,Desarrollador/a Back-end (Spring Boot),Requerimientos del cargo,<ul><li>Experiencia en desarrollo de solucione...,"<p>Somos un grupo de empresas de retail, espec...",<p><strong>¿Cuál será tu desafío en este cargo...,,,False,hybrid,...,lang_not_specified,Programming,"[wellness, accessible, meals_provided, commuti...",NaN,NaN,1716413884,NaN,NaN,33,https://www.getonbrd.com/jobs/desarrollador-a-...
3,machine-learning-engineer-neuralworks-santiago...,Machine Learning Engineer,Calificaciones clave,<ul><li>Ingeniería Civil en Computación o simi...,<div>NeuralWorks es una compañía de alto creci...,<div>El equipo de analytics trabaja en diferen...,<ul><li>MacBook Air M1 o similar (con opción d...,<ul><li>Experiencia en Infraestructura as code...,False,hybrid,...,es,Machine Learning & AI,"[library, accessible, flexible_hours, internal...",NaN,NaN,1716241209,NaN,NaN,54,https://www.getonbrd.com/jobs/machine-learning...
4,arquitecto-java-bilingue-kibernum-bogota,Arquitecto Java Bilingüe,Requerimientos del cargo,<p><em>Requisitos</em></p><ol><li>Profesional ...,"<p><strong>Kibernum</strong>, es una de las em...","<p>Responsable del diseño, arquitectura y docu...",<ul><li>Tipo de Trabajo: Remoto </li><li>Tipo ...,,True,remote_local,...,en,Programming,[computer_provided],NaN,NaN,1715899810,NaN,NaN,9,https://www.getonbrd.com/jobs/arquitecto-java-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,analista-qa-automatizador-iconstruye-remote,Analista QA Automatizador,Requisitos,<ul><li>Conocimientos de Ingeniería del Softwa...,<p>¡Súmate a nuestro equipo! Estamos buscando ...,"<ul><li>Analizar Requerimientos, Historias de ...",<ul><li>5 días extras de descanso.</li><li>Tar...,,False,hybrid,...,lang_not_specified,SysAdmin / DevOps / QA,"[flexible_hours, computer_provided, informal_d...",NaN,NaN,1715187574,NaN,NaN,16,https://www.getonbrd.com/jobs/analista-qa-auto...
96,full-stack-web-developer-remote-integrated-com...,Senior JavaScript Full-Stack Web Developer,Qualifications and requirements,<ul><li><strong>A 4 or 5-year traditional Bach...,<div><strong>About ICS &amp; the role</strong>...,<ul><li>Utilize expert knowledge of JavaScript...,"<ul><li>Opportunity to work on complex, creati...",<ul><li>Experience with Client-side Database m...,True,remote_local,...,en,Programming,"[flexible_hours, informal_dresscode]",3800.0,4800.0,1715187249,NaN,NaN,90,https://www.getonbrd.com/jobs/full-stack-web-d...
97,back-end-developer-remote-integrated-computer-...,Senior Back-end Web Developer,Qualifications and requirements,<ul><li><strong>A 4 or 5-year traditional Bach...,<div><strong

### cuantas categorias hay en el data set

In [31]:
unique_categories = df_jobs['category_name'].unique()

print("Categorias:", unique_categories)

print("Numero de categorias categorias:", len(unique_categories))


Categorias: ['SysAdmin / DevOps / QA' 'Programming' 'Machine Learning & AI'
 'Mobile Development' 'Data Science / Analytics'
 'Product, Innovation & Agile' 'Customer Support' 'Operations / Admin']
Numero de categorias categorias: 8
